In [1]:
from dask.distributed import Client, config
client = Client()
client

/usr/local/lib/python3.6/site-packages/distributed/client.py:1134: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | None   | 1.9.2     | None    |
| lz4     | None   | 3.1.1     | None    |
| msgpack | 1.0.2  | 1.0.0     | 1.0.2   |
+---------+--------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tcp://my-dask-scheduler:8786 Dashboard: http://my-dask-scheduler:8787/status,Cluster Workers: 3 Cores: 3 Memory: 10.50 GB


Execute this in a terminal tab

```
scp -r lheinric@lxplus.cern.ch:~/.globus ~/;
openssl rsa -in .globus/userkey.pem -out userkey.pem;
mv userkey.pem .globus/;
chmod 400 ~/.globus/userkey.pem;
```

In [2]:
import os
import rucio.client
os.environ['X509_USER_CERT'] = '~/.globus/usercert.pem'
os.environ['X509_USER_KEY']  = '~/.globus/userkey.pem'
os.environ['RUCIO_ACCOUNT']  = 'lheinric'
rucio_client = rucio.client.Client()

In [3]:
def get_cern_names():
    g = rucio_client.list_replicas([{'scope': 'data17_13TeV', 'name': 'data17_13TeV.periodK.physics_Main.PhysCont.DAOD_PHYSLITE.grp17_v01_p4309'}])
    cern_fnames = []
    for n,a in enumerate(g):
        cern_fname = [k for k,v in a['pfns'].items() if v['rse'] == 'CERN-PROD_LOCALGROUPDISK']
        cern_fnames.append(cern_fname[0])
        if n > 10:
            break
    return cern_fnames
fnames = get_cern_names()
fnames[:3]

['root://eosatlas.cern.ch:1094//eos/atlas/atlaslocalgroupdisk/dq2/rucio/data17_13TeV/26/59/DAOD_PHYSLITE.22958105._000001.pool.root.1',
 'root://eosatlas.cern.ch:1094//eos/atlas/atlaslocalgroupdisk/dq2/rucio/data17_13TeV/48/73/DAOD_PHYSLITE.22958105._000002.pool.root.1',
 'root://eosatlas.cern.ch:1094//eos/atlas/atlaslocalgroupdisk/dq2/rucio/data17_13TeV/63/bb/DAOD_PHYSLITE.22958105._000003.pool.root.1']

In [4]:
!voms-proxy-init --voms atlas

Contacting lcg-voms2.cern.ch:15001 [/DC=ch/DC=cern/OU=computers/CN=lcg-voms2.cern.ch] "atlas"...
Remote VOMS server contacted succesfully.


Created proxy in /tmp/x509up_u1000.

Your proxy is valid until Wed Mar 24 23:38:38 UTC 2021


In [9]:
pfilename = '/tmp/x509up_u1000'
proxydata = open(pfilename).read()

def run(fname,a):
    import os
    import random
    import time
    import uproot
    time.sleep(random.randint(2,5))
    with open(pfilename,'w') as pfile:
        pfile.write(proxydata)
    os.environ['X509_USER_PROXY'] = pfilename
    d = uproot.open(f'{fname}:CollectionTree')
    branches = list([b.name for b in d.branches])[:2]
    return {'module': branches}

In [10]:
import numpy as np
futures = client.map(run,fnames,np.random.random(len(fnames)))
client.gather(futures)

[{'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']},
 {'module': ['ByteStreamEventInfo', 'TrigConfKeys']}]